In [1]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
prefix_input = 'detectron2-input'
prefix_output = 'detectron2-ouput'

# Upload data for training

In [32]:
!git clone https://github.com/facebookresearch/detectron2

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [ ]:
! ./upload_coco2017_to_s3.sh "sagemaker-us-east-2-553020858742" "detectron2/datasets/coco"

Create stage directory: /home/ec2-user/SageMaker/coco-2017-2020-03-31-18-56-41
--2020-03-31 18:56:41--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.184.211
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.184.211|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘/home/ec2-user/SageMaker/coco-2017-2020-03-31-18-56-41/train2017.zip’

/home/ec2-user/Sage 100%[===================>]  18.01G  36.3MB/s    in 9m 46s  

2020-03-31 19:06:27 (31.5 MB/s) - ‘/home/ec2-user/SageMaker/coco-2017-2020-03-31-18-56-41/train2017.zip’ saved [19336861798/19336861798]

./upload_coco2017_to_s3.sh: line 19: zips/train2017.zip: No such file or directory
Extracting /home/ec2-user/SageMaker/coco-2017-2020-03-31-18-56-41/train2017.zip

In [58]:
# TODO: delete
# upload directory with COCO dataset to S3 location

import os
import boto3

s3_resource = boto3.resource("s3", region_name="us-east-2")
root_path = os.getcwd()
data_path = os.path.join(root_path, "detectron2/datasets/coco")



for path, subdirs, files in os.walk(data_path):
    directory_name = path.replace(root_path+"/","")
    for file in files:
        print(os.path.join(root_path, directory_name, file))
        s3_resource.Bucket(bucket).upload_file(os.path.join(root_path, directory_name, file), directory_name+'/'+file)
        print(f"file {file} has been uploaded to S3")

/home/ec2-user/SageMaker/detectron2-sagemaker/detectron2/datasets/coco/annotations/person_keypoints_val2017_100.json
file person_keypoints_val2017_100.json has been uploaded to S3
/home/ec2-user/SageMaker/detectron2-sagemaker/detectron2/datasets/coco/annotations/instances_minival2014_100.json
file instances_minival2014_100.json has been uploaded to S3
/home/ec2-user/SageMaker/detectron2-sagemaker/detectron2/datasets/coco/annotations/person_keypoints_minival2014_100.json
file person_keypoints_minival2014_100.json has been uploaded to S3
/home/ec2-user/SageMaker/detectron2-sagemaker/detectron2/datasets/coco/annotations/instances_val2017_100.json
file instances_val2017_100.json has been uploaded to S3


In [4]:
f"s3://{bucket}/detectron2/datasets/coco/train"

's3://sagemaker-us-east-2-553020858742/detectron2/datasets/coco/train'

In [66]:
!pygmentize ./detectron2/datasets/prepare_for_tests.sh

#!/bin/bash -e
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved

# Download some files needed for running tests.

cd "${0%/*}"

BASE=https://dl.fbaipublicfiles.com/detectron2
mkdir -p coco/annotations

for anno in instances_val2017_100 \
  person_keypoints_val2017_100 \
  instances_minival2014_100 \
  person_keypoints_minival2014_100; do

  dest=coco/annotations/$anno.json
  [[ -s $dest ]] && {
    echo "$dest exists. Skipping ..."
  } || {
    wget $BASE/annotations/coco/$anno.json -O $dest
  }
done


# Train your model

In [14]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = "us-east-2"
container = "d2-sm-base"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, container)

d2 = sagemaker.estimator.Estimator(image,
                                   role=role,
                                   train_instance_count=2, 
                                   train_instance_type='ml.p3.16xlarge',
                                   train_volume_size=100,
                                   output_path="s3://{}/{}".format(sess.default_bucket(), prefix_output),
                                   sagemaker_session=sess)

d2.fit({'train':"s3://sagemaker-us-east-2-553020858742/balloon/train",
        'val':"s3://sagemaker-us-east-2-553020858742/balloon/val"}) 

2020-04-07 20:07:00 Starting - Starting the training job...
2020-04-07 20:07:02 Starting - Launching requested ML instances......
2020-04-07 20:08:31 Starting - Preparing the instances for training.........
2020-04-07 20:10:00 Downloading - Downloading input data
2020-04-07 20:10:00 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-07 20:13:06,080 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-07 20:13:06,160 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-04-07 20:13:06,164 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-04-07 20:13:06,167 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-04-07 20:13:06,167 sagemaker-containers INFO     Generating setup.cfg
2020-04-0